In [1]:
import os

files_needed = [
    {"thinkplot.py": "https://github.com/AkeemSemper/ml_data/raw/main/thinkplot.py"},
    {"thinkstats2.py": "https://github.com/AkeemSemper/ml_data/raw/main/thinkstats2.py"},
]
current_folder = os.getcwd()
for f in files_needed:
    for file_name, url in f.items():
        if not os.path.exists(file_name):
            print(f"Downloading {file_name}")
            os.system(f"curl {url} -o {current_folder}/{file_name}")

In [2]:
import numpy as np
import random
import thinkstats2
import thinkplot
from scipy import stats as ss

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

<h1>Quiz 3</h1>

Please fill in the bodies of the functions as specified. Please read the instructions closely and ask for clarification if needed. A few notes/tips:
<ul>
<li>Like all the functions we use, the function is a self contained thing. It takes in values as paramaters when called, and produces a return value. All of the inputs that may change should be in that function call, imagine your function being cut/pasted into some other file - it should not depend on anything outside of libraries that it may need. 
<li>Test your function with more than one function call, with different inputs. See an example in comments below the first question. 
<li>If something doesn't work, print or look at the varaibles window. The #1 skill that'll allow you to write usable code is the ability to find and fix errors. Printing a value out line by line so you can see how it changes, and looking for the step where something goes wrong is A-OK and pretty normal. It is boring. 
<li>Unless otherwise specified, you can use outside library functions to calculate things. 
</ul>

<h1>Test Data</h1>

You may notice there's no data specified or attached. You'll need to generate some test data if you want to test your functions. 

The easiest way to generate test data is to use some of the random functions to generate data that looks like what you need. Numpy random and scipy disributions .rvs functions are good places to look, we've also generated random data many times in the past. 

There is no specific requirement on what your data needs to be, it just needs to be good enough to test your function. If you pay attention to what exactly you're calculating and the criteria given, you should be able to create some suitable data for different tests. As an example, for the Hyp Test question, you need two sets of normal data. You can generate some in many ways, one is through scipy:
<ul>
<li>ss.norm.rvs(loc=0, scale=1, size=1, random_state=None)
</ul>
<p>
Since you're checking if there's a significant difference between the two groups, you'd likely want multiple sets of data - two that are very close, so they will not show a difference, and two that are not close, so they will show a difference. Think about what you are checking, then just make some data that will allow you to test that. 

This should not be extremely difficult to code nor should it be super time consuming, the commands are pretty simple and generating random varaibles is pretty similar for any distribution. There is some though involved in saying "what data do I need to check this?" That's something that is pretty important in general, if we are creating something we need to make sure that it works in general, not just one example. Critically, there are not specific sets of data you need - almost anything will work. It is only there to let your functions run and see if they are correct. You don't need to aim for "the perfect test data" or anything like that, just make some data in a list, if it needs to be of a certain distribution, use that dist to get it; if the distribution doesn't matter, just make something. 

In [ ]:
# --- Generate Test Data ---
np.random.seed(42) # Set seed for reproducibility

# 1. Data for Ski Question (Observed counts)

ski_observed = [150, 90, 30, 45, 85] 

# 2. Data for Hypothesis Testing (Normal distributions)

group_A = ss.norm.rvs(loc=50, scale=5, size=100)

group_B = ss.norm.rvs(loc=52, scale=5, size=100)

group_C = ss.norm.rvs(loc=50, scale=5, size=100)

# 3. Data for Flex Test (Non-normal data)
# Exponential distribution
non_normal_data = ss.expon.rvs(size=50)

# 4. Data for Grades
# Random integers between 40 and 100
student_grades = np.random.randint(40, 100, size=200)

print("Test data generated successfully.")

Test data generated successfully.


<h1>Ski on Chi - 10pts</h1>

You operate a ski hill, and over the years you've seen the distribution of skiers vs snowboarders vs snow skaters etc... change a bit. This is your first full open season since the pandemic hit. When you closed in early 2020, the distribution of your customer base was:
<ul>
<li>Skiers - 40%
<li>Snowboarders - 20%
<li>Snow Skaters - 5%
<li>Non-Active (i.e. sit in the lodger) - 15%
<li>Lesson takers - 20%
</ul>

You are seeing a different pattern now, but you are not sure if that is due to a change in what your customers want or due to just random chance. You want to be able to analytically tell if what you observe each week is a real change from that baseline above, or nothing to worry about. 

In this function you'll take in:
<ul>
<li>Two list of values for the observed number of customers in each group, in the order indicated above. E.g. [35,25,10,10,20].
<li>An alpha value (the cutoff criteria for a p-values)
</ul>
<br><br>
You'll return 3 results:
<ul>
<li>A true/false assessment for if the data appears to show a significant difference in means, measured by if the pValue is less than the supplied alpha. 
<li>The name of the category that MOST EXCEEDS the expectation. 
<li>The name of the cetegory that is MOST EXCEEDED BY the expectation. 
</ul>

In [ ]:
def skiCustomersChange(observedCustys, alpha=.05):
    # Define the expected distribution percentages based on the prompt
    
    expected_ratios = [0.40, 0.20, 0.05, 0.15, 0.20]
    category_names = ["Skiers", "Snowboarders", "Snow Skaters", "Non-Active", "Lesson takers"]
    
    # Calculate expected counts based on the total number of observed customers
    total_observed = sum(observedCustys)
    expected_counts = [ratio * total_observed for ratio in expected_ratios]
    
    # Perform Chi-Squared Goodness of Fit Test
    chi_stat, p_value = ss.chisquare(f_obs=observedCustys, f_exp=expected_counts)
    
    # 1. Determine if significant difference exists
    isSignificantDiff = p_value < alpha
    
   
    differences = np.array(observedCustys) - np.array(expected_counts)
    
    # 2. Find category that MOST EXCEEDS expectation (Largest positive difference)
    highest_diff_index = np.argmax(differences)
    higherThanExp = category_names[highest_diff_index]
    
    # 3. Find category that is MOST EXCEEDED BY expectation (Largest negative difference)
    # We want the minimum value (most negative)
    lowest_diff_index = np.argmin(differences)
    lowerThanExp = category_names[lowest_diff_index]

    return isSignificantDiff, higherThanExp, lowerThanExp

In [6]:
# Test 1: Using the data generated above
is_sig, high, low = skiCustomersChange(ski_observed, alpha=0.05)
print(f"Test 1 Results:")
print(f"Significant Difference: {is_sig}")
print(f"Category higher than expected: {high}")
print(f"Category lower than expected: {low}")
print("-" * 30)

# Test 2: Using the specific example from the prompt instructions
# [35, 25, 10, 10, 20] vs expected percentages
is_sig_2, high_2, low_2 = skiCustomersChange([35, 25, 10, 10, 20], alpha=0.05)
print(f"Test 2 (Prompt Example):")
print(f"Significant Difference: {is_sig_2}")
print(f"Category higher than expected: {high_2}")
print(f"Category lower than expected: {low_2}")

Test 1 Results:
Significant Difference: True
Category higher than expected: Snowboarders
Category lower than expected: Non-Active
------------------------------
Test 2 (Prompt Example):
Significant Difference: False
Category higher than expected: Snowboarders
Category lower than expected: Skiers


<h2>Hypothesis Testing - 10pts</h2>

In this function you'll take in:
<ul>
<li>Two list of values - dataA and dataB. The data will be normally distributed. 
<li>An alpha value (the cutoff criteria for a p-values)
<li>A power value (the likelihood of not getting a false negative)
<li>An effect size value.
</ul>
<br><br>
You'll produce a tuple of 3 results:
<ul>
<li>A true/false assessment for if the data appears to show a significant difference in means, measured by if the pValue is less than the supplied alpha in a t-test.
<li>A true/false assessment for if a hypothesis test has enough power to be reliable, measured by if the power you calculate is greater than the supplied power. 
<li>A true false assessment for if the data appears to show a significant difference in means, measured by if the Cohen effect size is greater than the supplied effect size. 
</ul>

<b>Please report your responses in the format indicated in the template. As well, please report all true/false values as 1/0. 1 is True, 0 is false. To verify if all the criteria are true, someone calling this function should be able to multiply the 3 values in the tuple together and get a result of 1 if they are all true, and 0 otherwise</b>

In [7]:
from statsmodels.stats.power import TTestIndPower

def strengthOfEffect(dataA, dataB, alpha=.05, power=.8, effectSize=.5):
    
    # 1. T-Test for significance
    t_stat, p_val = ss.ttest_ind(dataA, dataB)
    passedPtest = 1 if p_val < alpha else 0
    
    # 2. Calculate Cohen's d (Effect Size)
    mean1, mean2 = np.mean(dataA), np.mean(dataB)
    n1, n2 = len(dataA), len(dataB)
    var1, var2 = np.var(dataA, ddof=1), np.var(dataB, ddof=1)
    
    # Calculate Pooled Standard Deviation
    pooled_sd = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
    cohens_d = abs(mean1 - mean2) / pooled_sd
    
    passedEffectSize = 1 if cohens_d > effectSize else 0
    
    # 3. Power Analysis
    # Calculate the power of the test based on the calculated effect size
    power_analysis = TTestIndPower()
    ratio = n2 / n1 # Ratio of sample sizes
    calculated_power = power_analysis.solve_power(effect_size=cohens_d, 
                                                  nobs1=n1, 
                                                  alpha=alpha, 
                                                  power=None, 
                                                  ratio=ratio)
    
    passedPower = 1 if calculated_power > power else 0

    results = (passedPtest, passedPower, passedEffectSize)
    return results

In [10]:
# Test 1: Compare Group A vs Group B (Should likely be different)
results_AB = strengthOfEffect(group_A, group_B, alpha=0.05, power=0.8, effectSize=0.3)
print(f"A vs B Results (P-Test, Power, EffectSize): {results_AB}")

# Test 2: Compare Group A vs Group C (Should likely NOT be different)
results_AC = strengthOfEffect(group_A, group_C, alpha=0.05, power=0.8, effectSize=0.5)
print(f"A vs C Results (P-Test, Power, EffectSize): {results_AC}")

A vs B Results (P-Test, Power, EffectSize): (1, 1, 1)
A vs C Results (P-Test, Power, EffectSize): (0, 0, 0)


<h2>Safe Test - 10pts</h2>

In this function you'll take in:
<ul>
<li>Two list of values - dataA and dataB.
</ul>
<br><br>
You'll produce a p-value for a two sided hypothesis test:
<ul>
<li>If the data is not normally distributed, use a Mann-Whitney Test. 
<li>If the data appears to be normally distributed, and the variance differs substantially, use a Welch's t-test.
<li>If none of those conditions are true, use a 'normal' (Student's) t-test. 
<li>Note: The execution of all of these tests are very similar from your persepective. They are all in the scipy documentation - Google for exact details, and the code closely mirrors the examples we did. 
<li>Note 2: If you ever need to use a cutoff for a p-value in the middle of your calculations, please choose something reasonable. There are common defaults for whatever you may need. These defaults are likely shown in the documentation or any examples you may look up. 
</ul>

<b>In any case, the value returned is one number (not in a list, tuple, etc...) that is the pValue performed for that test. 

In [ ]:
def flexHypTest(dataA, dataB):
    # 1. Check Normality (using Shapiro-Wilk)
    
    statA, p_normA = ss.shapiro(dataA)
    statB, p_normB = ss.shapiro(dataB)
    
    # Cutoff for normality check (standard is 0.05)
    normality_cutoff = 0.05
    
    if p_normA < normality_cutoff or p_normB < normality_cutoff:
        # Data is NOT normal -> Use Mann-Whitney U Test
        stat, pValue = ss.mannwhitneyu(dataA, dataB, alternative='two-sided')
    else:
        
        statL, p_levene = ss.levene(dataA, dataB)
        
        if p_levene < 0.05:
            # Variances differ substantially -> Welch's t-test (equal_var=False)
            stat, pValue = ss.ttest_ind(dataA, dataB, equal_var=False)
        else:
            # Normal and Equal Variances -> Student's t-test (equal_var=True)
            stat, pValue = ss.ttest_ind(dataA, dataB, equal_var=True)
    
    return pValue

In [12]:
# Test 1: Normal Data (A vs B) - Should use T-Test
p_val_normal = flexHypTest(group_A, group_B)
print(f"P-Value (Normal Data): {p_val_normal}")

# Test 2: Non-Normal Data (A vs Non-Normal) - Should use Mann-Whitney
p_val_non_normal = flexHypTest(group_A, non_normal_data)
print(f"P-Value (Non-Normal Data): {p_val_non_normal}")

P-Value (Normal Data): 9.10654008630498e-05
P-Value (Non-Normal Data): 2.172254270994724e-23


<h1>Grade Distribution - 10pts</h1>

Grade distributions for final letter grades at a school are generally skewed towards the higher end of the scale. We can model it with a function below.

Percentage grades on individual assignments are often skewnormally distributed. (Note: this is more for curved schools than somewhere like NAIT with hard cutoffs. When I was in school CompSci profs would aim for a 50%-60% raw average to get a normal-ish distribution of marks.)

You are seeking to generate a grading system, in two steps:
<ul>
<li>Use the supplied Weibull distribution in the simpleGenerateLetterGradeBuckets function to generate the distribution of letter grades - A,B,C,D,F. We are a simple school and we only have letters, no plus or minus. 
<li>
<li>Use the function simpleGenerateLetterGradeBuckets to tell you HOW MANY slots there are for each grade. This is done for you in the provided function, you just need to call it and get the results. Please pay attention to the n value for number.
<li>Take the supplied raw percentage grades and fit them into those buckets. I.E. if there are 17 slots for an A grade, the 17 highest percentage marks should get an A; if there are then 52 for B, then the next 52 highest get a B, etc...
<li><b>You are going to return a list of tuples - the original percentage grade, and the letter grade. E.g. [(72,B), (84,A), etc...]</b>
</ul>

<br><br>
In this function you'll take in:
<ul>
<li>A list of raw percentage grades, from 0 to 100. E.g. [100,98,24,53,45, etc...]
</ul>

You'll produce:
<ul>
<li>A list of tuples. Each tuple is the original percentage grade, and the letter grade. .
</ul>

<br>
Note: You'll have to run the function cell down at the bottom first. 
<br><br>
<b>Bonus: The provided function for grade buckets probably isn't the best overall, if you can rewrite it to be better, up to 3 bonus marks. Think about the random factor...</b>

In [ ]:
def assignLetterGrades(rawPercentageGrades):
    # 1. Get the bucket counts for the letter grades
    n = len(rawPercentageGrades)
    buckets = simpleGenerateLetterGradeBuckets(n)
    # buckets example: {'A': 10, 'B': 20, ...}
    
    # 2. Prepare grades for sorting while keeping track of original index
  
    indexed_grades = []
    for i, grade in enumerate(rawPercentageGrades):
        indexed_grades.append((grade, i))
        
    # 3. Sort grades High -> Low
    indexed_grades.sort(key=lambda x: x[0], reverse=True)
    
    # 4. Assign letters based on bucket counts
    final_results = []
    grade_order = ['A', 'B', 'C', 'D', 'F']
    
    current_student_idx = 0
    
    for letter in grade_order:
        count = buckets[letter]
       
        for _ in range(count):
            if current_student_idx < len(indexed_grades):
                grade_val, original_idx = indexed_grades[current_student_idx]
                # Store as (original_index, grade_val, letter)
                final_results.append((original_idx, grade_val, letter))
                current_student_idx += 1
                
    # 5. Sort back by original index to maintain the list order 
   
    final_results.sort(key=lambda x: x[0])
    
    # Format as requested: [(Percentage, Letter), ...]
    listOfTumples = [(x[1], x[2]) for x in final_results]

    return listOfTumples

In [23]:
#Example for 423 students
simpleGenerateLetterGradeBuckets(423)

{'A': 65, 'B': 173, 'C': 121, 'D': 46, 'F': 18}

In [ ]:
def simpleGenerateLetterGradeBuckets(n=100):
    #Define distribution params
    c = 1.5
    loc = 3
    scale = 1.5

    #Generate distribution buckets
    aGrades = 0
    bGrades = 0
    cGrades = 0
    dGrades = 0
    fGrades = 0

    #Define cutoffs - count above cut are grade slots. 
    cuts = [3.7, 2.9, 1.9, .9]
    data = 7.2-ss.weibull_min.rvs(c, loc, scale, n)
    
    #Count the number of slots for each letter grade
    for i in range(len(data)):
        tmp = data[i]
        if tmp > cuts[0]:
            aGrades += 1
        elif tmp > cuts[1]:
            bGrades += 1
        elif tmp > cuts[2]:
            cGrades += 1
        elif tmp > cuts[3]:
            dGrades += 1
        else:
            fGrades += 1
    buckets = {"A":aGrades, "B":bGrades, "C":cGrades, "D":dGrades, "F":fGrades}
    return buckets

In [15]:
# Call the function with the random grades we generated
final_grades = assignLetterGrades(student_grades)

# Print the first 10 results to verify
print("First 10 Student Grades (Raw, Letter):")
for item in final_grades[:10]:
    print(item)

# Optional: Print a count of how many of each letter were assigned to verify distribution
from collections import Counter
letters = [x[1] for x in final_grades]
print("\nTotal Counts per Letter:")
print(Counter(letters))

First 10 Student Grades (Raw, Letter):
(np.int32(81), 'B')
(np.int32(48), 'C')
(np.int32(89), 'A')
(np.int32(66), 'B')
(np.int32(97), 'A')
(np.int32(41), 'D')
(np.int32(44), 'D')
(np.int32(68), 'B')
(np.int32(76), 'B')
(np.int32(77), 'B')

Total Counts per Letter:
Counter({'B': 85, 'C': 50, 'A': 42, 'D': 17, 'F': 6})
